In [1]:
import pickle
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']
ACTION_TO_INT = {'UP':0, 'RIGHT':1, 'DOWN':2, 'LEFT':3, 'WAIT':4, 'BOMB':5}

In [2]:
# TODO: check put PyTables to save and load entries of large dictionaries saved as a file
# TODO: implement feature is_crate_in_bomb_range(creates, bombs)
# TODO 

In [3]:
d = dict()
state = np.array([0, 0])
k = tuple(state.tolist())   # convert numpy array to a tuple

d[k] = [[],[]]
d[k][0] = 0
d[k][1] = 1
d[k]

new_state = np.asarray(k)   # convert tuple back to numpy array
new_state

array([0, 0])

In [4]:
with open("my-saved-q-table.pt", "rb") as file:
    q_table:dict = dict(pickle.load(file))
print("q-table entries", len(q_table))

q-table entries 3020


In [5]:
with open("stats.pt", "rb") as file:
    stats:dict = dict(pickle.load(file))
print(stats)

{'n_games': 256478}


In [6]:
def state_to_action_via_lin_val_est(state_str:str):
    temp_str = state_str[1:-1]
    state = np.fromstring(temp_str, dtype=float, sep=' ')
    Q = np.zeros(len(ACTIONS))
    for i, a in enumerate(ACTIONS):
        Q[i] = np.dot(state, action_vectors[a])
    action_int = Q.argmax()
    return ACTIONS[action_int]

In [7]:
c = 0
for key in q_table.keys():
    #print(key)
    action = q_table[key].argmax()
    print(key, "simple_q_learning: ", ACTIONS[action] )
    print(q_table[key] )
    print()

59 -1.93656079  1.07464627 -1.39385574 -1.3947477 ]

(1.0, 0.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 3.0, 0.0, 3.0, 3.0, 3.0, 0.0, 3.0, 0.0, 1.0, -2.0, 0.0, 1.0, 0.0, 0.0) simple_q_learning:  LEFT
[1.01267484 1.05466307 0.62149849 1.07931424 1.03439095 1.04906132]

(1.0, 0.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 2.0, 0.0, 2.0, 2.0, 2.0, 0.0, 2.0, 0.0, 1.0, -2.0, 0.0, 1.0, 0.0, 0.0) simple_q_learning:  RIGHT
[1.00485447 1.05847208 0.51905118 1.04890467 1.02424863 1.00989812]

(1.0, 0.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, -2.0, 0.0, 1.0, 0.0, 0.0) simple_q_learning:  DOWN
[0.612612   1.00538791 1.0636349  1.00384756 1.0031389  1.006891  ]

(1.0, 0.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -2.0, 0.0, 1.0, 0.0, 0.0) simple_q_learning:  DOWN
[1.03115442 1.03339423 1.0979737  1.05915504 1.0780159  1.0737514 ]

(1.0, 1.0, -1.0, -1.0, 0.0, -1.0, 1.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0

In [8]:
a = np.array([[1,2,3], [4,5,6]])
a = np.reshape(a, 6)
print("a: ", a)

b = np.array([1, 2, 3])
print("b: ", b)
channels = np.concatenate((a, b))
stacked_channels = np.stack(channels)
stacked_channels.reshape(-1)
stacked_channels


a:  [1 2 3 4 5 6]
b:  [1 2 3]


array([1, 2, 3, 4, 5, 6, 1, 2, 3])

In [9]:
def get_danger_zone(bombs):
    # TODO: sort bombs by countdown (descending)
    field_len = 17
    range:int = 4   #3 tiles in each direction
    danger_zone = np.zeros((field_len, field_len))
    for p, t in sorted (bombs, key = lambda x: x[1], reverse=True):
        danger_zone[ max(p[0]-range,0):min(p[0]+range, field_len) , p[1]] = t
        danger_zone[ p[0], max(p[1]-range,0):min(p[1]+range, field_len)] = t
    return danger_zone

bombs = [((4, 4),3), ((3, 3),1), ((2, 2), 4)]
danger_zone = get_danger_zone(bombs)
print(danger_zone)

[[0. 0. 4. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 4. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [4. 4. 4. 1. 3. 4. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 3. 3. 1. 3. 3. 3. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 4. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
